In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import random
import os

In [ ]:
base = os.path.basename('/home/kylecshan/data/index/all/00002469b818f290.jpg')
os.path.splitext(base)[0]

In [ ]:
class ImageFolderWithID(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithID, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        base = os.path.basename(path)
        img_id = os.path.splitext(base)[0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (img_id,))
        return tuple_with_path

In [ ]:
input_size = 224
batch_size = 4

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'index': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
    
data_dir = '/home/kylecshan/data/images224/train_ms2000_v4/'
image_datasets = {x: ImageFolderWithID(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'index']}
# index_dataset = ImageFolderWithPaths(root='/home/kylecshan/data/images224/train_ms2000_v4/',
#                                            transform=data_transform)
# num_total = len(index_dataset)

# index_loader = torch.utils.data.DataLoader(index_dataset, batch_size=4, num_workers=4)

dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, num_workers=4) 
                    for x in ['train', 'index']}
    
    

In [ ]:
# print('test: %d' % (num_total))
print (len(dataloaders_dict['train'].dataset))
print (len(dataloaders_dict['index'].dataset))

data_index = []
count = 0
for i, l, p in dataloaders_dict['test']:
#     print (i)
    print (l.cpu().tolist())
#     data_index.extend(list(p))
    count += 1
    
    if count >= 100:
        break
    
# print (data_index)

In [ ]:
def show_landmarks_batch(sample_batched):
    """Show image with landmarks for a batch of samples."""
    images_batch, _, _ = sample_batched
    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

for i_batch, sample_batched in enumerate(dataloaders_dict['train']):
    if i_batch <= 5:
        plt.figure()
        show_landmarks_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
    else:
        break

In [ ]:
import sys
sys.path.insert(0,'/home/kylecshan/Kaushik/retrieval')
from model import initialize_model

# model_path = '/home/kylecshan/saved/05_16_2019/model815.pth'

# model_name = "densenet169"

model_path = '/home/kylecshan/saved/05_15_2019/model3.pth'

model_name = "resnet50"

model_ft = initialize_model(model_name, use_pretrained=True)
model_ft_dict = model_ft.state_dict()

model = torch.nn.Sequential(*(list(model_ft.children())[:-1]))
#model = nn.DataParallel(model)
model_dict = model.state_dict()
# model_dict = model.named_parameters()

load_dict = torch.load(model_path, map_location='cpu')

# count = 0
# for k, v in load_dict.items():
#     print (k)
#     count += 1
#     if count >= 5:
#         break


# temp_dict = dict(load_dict)
# for k, v in temp_dict.items():
#     s = list(k)
#     del s[:15]
#     s.insert(0, "0") 
#     newk = "".join(s)
#     load_dict[newk] = load_dict.pop(k)
    
values = []    
for k, v in load_dict.items():
    values.append(v)

for lv, (mk, mv) in zip(values[:-2], model_dict.items()):
    mv = lv
    
# count = 0
# for k, v in load_dict.items():
#     print (k)
#     count += 1
# #     if count >= 5:
# #         break

# model_dict = {k: v for k, v in load_dict.items() if k in model_dict}

# model.load_state_dict(model_dict)

count = 0
for k, v in model_ft_dict.items():
    print (k)
    count += 1
    if count >= 5:
        break
print ("-"*80)       
count = 0    
for k, v in model_dict.items():
    print (k)
    count += 1
#     if count >= 5:
#         break
# print ("-"*80)       
# print (list(model.children()))      
# print ("-"*80)       
# print (list(model_ft.children()))


# print (list(model_ft.children())[:-1])
# print (list(model_ft.children()))

In [ ]:
import csv


def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:2] for line in csvreader]
    return key_url_list[1:]  # Chop off header

test_file = "/home/kylecshan/data/test.csv"

In [ ]:
key_url_test = ParseData(test_file)
print(key_url_test[32])

In [ ]:
import requests
import time

count = 0
start = time.time()

for _, url in key_url_test[900:1000]:
    if url != 'None':
        request = requests.get(url)
        if request.status_code != 200:
            count += 1
    else:
        count += 1
    
end = time.time()
        
print (end - start, count)

In [ ]:
import http.client as http
import time

count = 0
start = time.time()


for _, url in key_url_test[:100]:
    if url != 'None':
        c = http.InvalidURL(url)
#         c.request("HEAD", '')
#         if c.getresponse().status != 200:
#             count += 1
#     else:
#         count += 1
    
# end = time.time()
        
# print (end - start, count)

In [ ]:
import urllib3
import time

count = 0
start = time.time()

for _, url in key_url_test[900:1000]:
    if url != 'None':
        try:
            response = urllib3.urlopen(url)
            print(response.code)
        except:
            response = urllib3.urlopen(url)
            count += 1         
    else:
        count += 1
    
end = time.time()
        
print (end - start, count)

In [ ]:
import faiss  
import numpy as np # make faiss available
index = faiss.IndexFlatL2(10)   # build the index, d=size of vectors 
a = np.eye(10).astype('float32')
# here we assume xb contains a n-by-d numpy matrix of type float32
for i in range(3):
    index.add(a)                  # add vectors to the index
    print (index.ntotal)
    
    
k = 4                          # we want 4 similar vectors
D, I = index.search(a, k)     # actual search
print (D, I)
